# Crop Loss Prediction Project

In [ ]:
# Imports
import descarteslabs as dl
from datetime import datetime
import sys
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
from datetime import date
import shapefile
import numpy as np

In [ ]:
# Methods
def filter_duplicates(items):
    # Removing duplicate consecutive coordinates in a polygon
    for item in items:
        prev = None
        for i in item:
            if i != prev:
                yield i 
                prev = i

            
def search_scenes(geom, product, start_datetime, end_datetime):
    # Searching for scenes of a specific satellite product over `geom` and within the datetimes entered
    scenes, ctx = dl.scenes.search(geom,
                                   products=product,
                                   start_datetime = start_datetime,
                                   end_datetime = end_datetime
                                   )    
    return scenes, ctx

def read_aoi_shapefile(filename):
    # Reading the shapefile with all AOIs, and convert them to a dataframe
    shapes = shapefile.Reader(filename)
    aois = shapes.shapeRecords()
    col_names = ["y3_hhid", "gardenid", "plotid", "coordinates"]
    df = pd.DataFrame(index=np.arange(0, len(aois)), columns = col_names)
    for i, r in enumerate(aois):
        df.at[i] = [r.record[0], r.record[1], r.record[2], np.nan]
        df.at[i, "coordinates"] = [r.shape.__geo_interface__['coordinates'][0]]
    return df

def extract_sat_reflectance(lsms_df,
                            aois_df, 
                            product_name,
                            bands,
                            resolution,
                            start_datetime,
                            end_datetime):
    
    
    samples_df = pd.DataFrame(columns = lsms_df.columns)

    geom = {
        "type": "Polygon",
        "coordinates": [
        ]
    }
    
    # Looping over AOI polygons
    for i_plot in range(0, 20):#LSMS_df.shape[0]):
        y3_hhid = lsms_df["y3_hhid"][i_plot]
        gardenid = lsms_df["gardenid"][i_plot]
        plotid = lsms_df["plotid"][i_plot]

        aoi = aois_df.loc[(aois_df['y3_hhid'] == y3_hhid)
                    & (aois_df['gardenid'] == gardenid)
                    & (aois_df['plotid'] == plotid)]
        
        if not aoi.empty:
            coors = list()
            coor = aoi.iloc[0]["coordinates"]
            coors.insert(0, tuple(filter_duplicates(coor)))
            geom['coordinates'] = coors
            scenes, ctx = search_scenes(geom, product_name, start_datetime, end_datetime)

            ctx.assign(resolution = resolution)
            arr = scenes[-1].ndarray(bands[0], ctx)

            sample = lsms_df.iloc[i_plot].copy()
            sample.loc['datetime'] = list(scenes.each.properties['acquired'])
            
            # Eaxtracing band information for each scene
            for band in bands:
                data = np.empty((arr.data.shape[1], arr.data.shape[2], len(scenes)))
                for scene in range(0, len(scenes)):
                    band_scale = list(scenes.each.properties.bands.blue.data_range)[0][1]
                    arr = scenes[scene].ndarray(band, ctx)
                    band_data = arr.data / band_scale
                    band_data[arr.mask] = np.nan
                    data[:, :, scene] = np.squeeze(band_data.swapaxes(0, 1).swapaxes(1, 2))#.reshape(-1, arr.data.shape[0])
                sample.loc[band] = data
            
            
            # Extracing cloud mask
            data = np.empty((arr.data.shape[1], arr.data.shape[2], len(scenes)))
            for scene in range(0, len(scenes)):
                arr = scenes[scene].ndarray('cloud-mask', ctx)
                band_data = arr.data
                data[:, :, scene] = np.squeeze(band_data.swapaxes(0, 1).swapaxes(1, 2))#.reshape(-1, arr.data.shape[0])
            sample.loc['cloud-mask'] = data

            samples_df = samples_df.append(sample)
    
    # Removing columns with geolocation info
    samples_df = samples_df.drop(columns=["y3_hhid", "gardenid", "plotid", "hh_gps_lat", "hh_gps_long"])

    return samples_df

## Malawi Case Study

### Reading survery data 

In [ ]:
lsms_malawi_filename = '/home/ec2-user/crop-loss-EPAR/data/Malawi Survey Responses EPAR Radiant 2Sigma.csv'
lsms_malawi_df = pd.read_csv(lsms_malawi_filename)
lsms_malawi_df.head()

### Reading AOIs 

In [ ]:
# Reading the shapefile to a dataframe with three IDs, and coordinates
# Data cleanings that are done outside of this notebook:
# Duplicate entries are cleaned, missing data are removed, and 

aoi_malawi_filename = '/home/ec2-user/crop-loss-EPAR/data/Malawi_LSMS_Enumeration_Areas_100m_Example.shp'
aois_malawi_df = read_aoi_shapefile(aoi_malawi_filename)

### Extracting Satellite Imagery data 

In [ ]:
malawi_bands = ["blue", "green", "red", "nir", "swir1", "swir2"]
malawi_start = "2015-12-01"
malawi_end = "2016-04-30"

malawi_df = extract_sat_reflectance(lsms_df = lsms_malawi_df,
                        aois_df = aois_malawi_df,
                        product_name = "landsat:LC08:PRE:LaSRC",
                        bands = malawi_bands,
                        resolution = 30,
                        start_datetime = malawi_start,
                        end_datetime = malawi_end)

In [ ]:
malawi_df.head()

### Export data to csv 

In [ ]:
malawi_df.to_csv('/home/ec2-user/crop-loss-EPAR/exports/malawi_data.csv')